# 123 实现一个路线规划编制程序
在这个项目中，你将使用`A*`搜索实现一个“Google-maps”风格的路线规划算法。

In [34]:
# Run this cell first!

from helpers import Map, load_map, show_map
from helper import Maps, load_maps, show_maps
from student_code import shortest_path

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Map 基础

In [35]:
map_10 = load_map('map-10.pickle')
show_map(map_10)

上面的地图（如果你没有看到，请运行代码单元格）

**如果由于网络问题，仍然不显示地图**
**请运行show_maps(map_10)，并单击左上角Jupyter回到上级菜单，下载.html文件既可浏览。**

显示了一个包含10个交叉点的不连贯网络。左边的两个交叉口相互连接，但它们没有连接到道路网络的其余部分。在上面的图表中，2个节点（交叉点）之间的边缘代表一条文字直线道路，而不仅仅是2个城市的抽象连接。

这些`Map`对象有两个属性，你可以使用它们来实现A *搜索：`intersections`和`roads` 。

**Intersections**

`intersections`被表示为一个词典。

在这个例子中，有10个交点，每个交点用x、y坐标标识。坐标如下所示。你可以将鼠标悬停在上图中的每个点上查看交叉点编号。

In [36]:
map_10.intersections

{0: [0.7798606835438107, 0.6922727646627362],
 1: [0.7647837074641568, 0.3252670836724646],
 2: [0.7155217893995438, 0.20026498027300055],
 3: [0.7076566826610747, 0.3278339270610988],
 4: [0.8325506249953353, 0.02310946309985762],
 5: [0.49016747075266875, 0.5464878695400415],
 6: [0.8820353070895344, 0.6791919587749445],
 7: [0.46247219371675075, 0.6258061621642713],
 8: [0.11622158839385677, 0.11236327488812581],
 9: [0.1285377678230034, 0.3285840695698353]}

**Roads**

`roads`属性是一个列表，如果`i` 是一个交叉点，`roads[i]`包含一个交叉点`i`连接到的所有交叉点列表。

In [37]:
# this shows that intersection 0 connects to intersections 7, 6, and 5
map_10.roads[0] 

[7, 6, 5]

In [38]:
# This shows the full connectivity of the map
map_10.roads

[[7, 6, 5],
 [4, 3, 2],
 [4, 3, 1],
 [5, 4, 1, 2],
 [1, 2, 3],
 [7, 0, 3],
 [0],
 [0, 5],
 [9],
 [8]]

In [39]:
# map_40 is a bigger map than map_10
map_40 = load_map('map-40.pickle')
show_map(map_40)

In [40]:
map_40.roads

[[36, 34, 31, 28, 17],
 [35, 31, 27, 26, 25, 20, 18, 17, 15, 6],
 [39, 36, 21, 19, 9, 7, 4],
 [35, 20, 15, 11, 6],
 [39, 36, 21, 19, 9, 7, 2],
 [32, 16, 14],
 [35, 20, 15, 11, 1, 3],
 [39, 36, 22, 21, 19, 9, 2, 4],
 [33, 30, 14],
 [36, 21, 19, 2, 4, 7],
 [31, 27, 26, 25, 24, 18, 17, 13],
 [35, 20, 15, 3, 6],
 [37, 34, 31, 28, 22, 17],
 [27, 24, 18, 10],
 [33, 30, 16, 5, 8],
 [35, 31, 26, 25, 20, 17, 1, 3, 6, 11],
 [37, 30, 5, 14],
 [34, 31, 28, 26, 25, 18, 0, 1, 10, 12, 15],
 [31, 27, 26, 25, 24, 1, 10, 13, 17],
 [21, 2, 4, 7, 9],
 [35, 26, 1, 3, 6, 11, 15],
 [2, 4, 7, 9, 19],
 [39, 37, 29, 7, 12],
 [38, 32, 29],
 [27, 10, 13, 18],
 [34, 31, 27, 26, 1, 10, 15, 17, 18],
 [34, 31, 27, 1, 10, 15, 17, 18, 20, 25],
 [31, 1, 10, 13, 18, 24, 25, 26],
 [39, 36, 34, 31, 0, 12, 17],
 [38, 37, 32, 22, 23],
 [33, 8, 14, 16],
 [34, 0, 1, 10, 12, 15, 17, 18, 25, 26, 27, 28],
 [38, 5, 23, 29],
 [8, 14, 30],
 [0, 12, 17, 25, 26, 28, 31],
 [1, 3, 6, 11, 15, 20],
 [39, 0, 2, 4, 7, 9, 28],
 [12, 16, 22, 

**如果由于网络问题，仍然不显示地图**
**请运行show_maps(map_40)，并单击左上角Jupyter回到上级菜单，下载.html文件既可浏览。**

### 高级可视化

上图显示了一个跨越40个不同交叉点的道路网络（标记为0到39）。

生成此地图网络的`show_map`函数还需要一些可选参数，这些参数对于将你要编写的搜索算法的输出可视化可能很有用。

* `start` - 搜索算法的“开始”节点。
* `goal`  - “目标”节点。
* `path`  - 与地图上的有效交叉点访问序列相对应的整数数组。

In [41]:
# run this code, note the effect of including the optional
# parameters in the function call.
show_map(map_40, start=5, goal=34, path=[5,16,37,12,34])

### 编写你的算法
你应该在另一个选项卡中打开文件`student_code.py`并在那里编写你的算法。选择`File > Open`，然后选择上一句所说的这个文件。

你编写的算法将负责生成类似上面传入`show_map`的`path`。事实上，当调用相同的地图、起始点和目标点时，如上所述，算法应该生成路径`[5, 16, 37, 12, 34]` 。

In [42]:

%%bash
print(map_40.roads)
> shortest_path(map_40, 5, 34)
[5, 16, 37, 12, 34]

bash: line 1: syntax error near unexpected token `map_40.roads'
bash: line 1: `print(map_40.roads)'


In [ ]:
path = shortest_path(map_40, 8, 24)

In [96]:
#print(map_40.roads)
path = shortest_path(map_40, 8, 24)

if path == [8, 14, 16, 37, 12,17,10,24]:
    print("great! Your code works for these inputs!")
else:
    print("something is off, your code produced the following:")
    print(path)

shortest path called
great! Your code works for these inputs!


### 测试你的代码
如果下面的代码没有产生错误，那么你的算法的运行就是正确的。你马上就要提交项目了！在提交之前，请阅读以下提交清单：

**提交清单**

1. 我的代码是否通过了所有测试？
2. 我的代码是否实现了`A*`搜索，而不是其他搜索算法？
3. 我是否使用**可接受的启发式程序**来实现搜索目标？
4. 我使用的数据结构是否可以避免不必要的慢速查找？

当你对所有这些问题的回答都是“是的”时，请按下右下方的提交按钮进行提交！

In [97]:
from test import test

test(shortest_path)

shortest path called
shortest path called
shortest path called
All tests pass! Congratulations!
